In [12]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import math
import re
from collections import Counter

sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import configuration_util

In [2]:
def summarize_n_hypermutated_and_signatures_of_cases(mutClassificationDir):
    nCasesSummaryL = []
    signaturesSummaryL = []
    for f in os.listdir(mutClassificationDir):
        cancerType = re.sub('.tsv', '', f)
        filePath = os.path.join(mutClassificationDir, f)
        df = pd.read_table(filePath)
        hypermutatedDf = df[df['hypermutantClassification'] == 'Hypermutated']
        highMutBurdenDf = df[df['hypermutantClassification'] == 'highMutationBurden']
        
        nCasesSummaryL.append({'cancerType': cancerType,
        'nHypermutated': hypermutatedDf.shape[0], 'nTotal': df.shape[0], 'nHighMutBurden': highMutBurdenDf.shape[0]})
    
        #TODO make the code for plotting the signatures
        signatureCounter = Counter(hypermutatedDf['dominantSignature'])
        for signature, count in signatureCounter.items():
            signature = re.sub('mean_', '', signature)
            signaturesSummaryL.append({'cancerType': cancerType,
                'signature': signature, 'nCases': count, 
                'nHypermutatedCases': hypermutatedDf.shape[0], 'nTotal': df.shape[0]})
        
    casesSummaryDf = pd.DataFrame(nCasesSummaryL)
    signaturesSummaryDf = pd.DataFrame(signaturesSummaryL)
    return casesSummaryDf, signaturesSummaryDf
        

In [3]:
def summarize_dominant_signatures_of_cases(mutClassificationDir = '/Users/friedman/Desktop/hypermutationStatusIds/'):
    d = {}
    for f in os.listdir(mutClassificationDir):
        cancerType = re.sub('.tsv', '', f)
        filePath = os.path.join(mutClassificationDir, f)
        df = pd.read_table(filePath)
        hypermutatedDf = df[df['hypermutantClassification'] == 'Hypermutated']
        domSigDict = dict(zip(hypermutatedDf['Tumor_Sample_Barcode'], hypermutatedDf['dominantSignature']))
        for tsb, domSig in domSigDict.items():
            d[tsb] = domSig
    return d

In [ ]:
filePathDict = configuration_util.get_all_files_path_dict()

In [13]:
casesSummary, signaturesSummary = summarize_n_hypermutated_and_signatures_of_cases(
    mutClassificationDir = filePathDict['HYPERMUTATION_STATUS_IDS'])

In [16]:
dominantSignatureDict = summarize_dominant_signatures_of_cases(
    mutClassificationDir = filePathDict['HYPERMUTATION_STATUS_IDS'])

In [17]:
#ADJUST THE CASES SUMMARY INFORMATION
minNCasesToDisplay = 250 #lump all cancers with fewer than this number of cases as other
casesSummary['cancerType'] = casesSummary.apply(lambda row: 'other' if row['nTotal'] < minNCasesToDisplay else row['cancerType'] ,axis=1)


In [18]:
#ADJUST THE SIGNATURES SUMMARY INFORMATION
minNHypermutatedCasesToDisplay = 10
signaturesSummary['cancerType'] = signaturesSummary.apply(
    lambda row: 'other' if row['nHypermutatedCases'] < minNHypermutatedCasesToDisplay else row['cancerType'] ,axis=1)
signaturesSummary['orderingVal'] = signaturesSummary.apply(
    lambda row: -1 if row['cancerType'] == 'other' else row['nHypermutatedCases'], axis=1)

signaturesRenameDict = {'1': 'MMR', 'SMOKING': 'SMOKING',
                        'MMR': 'MMR', 'APOBEC': 'APOBEC', '10': 'POLE',
                       '11': 'TMZ', '14': 'POLE&MMR', '7': 'UV'}
signaturesSummary['signature'] = signaturesSummary['signature'].apply(lambda x:
        signaturesRenameDict[x] if x in signaturesRenameDict else 'other')

In [19]:
cancersOfIndeterminateOrigin = set(['Cancer_of_Unknown_Primary', 'Skin_Cancer,_Non-Melanoma'])  #remove cancers of indeterminate origin
signaturesSummary = signaturesSummary[~signaturesSummary['cancerType'].isin(cancersOfIndeterminateOrigin)]
casesSummary = casesSummary[~casesSummary['cancerType'].isin(cancersOfIndeterminateOrigin)]


In [ ]:
writePath = os.path.join(os.get_cwd(), 'FIGURE1_PLOTTING_FILES')

In [9]:
signaturesSummary.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/figure1cSignatureSummary.tsv', index=False, sep='\t')
casesSummary.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/figure1bCancerTypeSummary.tsv', index=False, sep='\t')

In [ ]:
def get_per_case_nmut_mb_info(nmutDfPath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/nmutInfo_impact_filtered.tsv'):
    df = pd.read_table(nmutDfPath)
    return dict(zip(df['Tumor_Sample_Barcode'], df['Nmut_Mb']))